In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32
3,0,99,3.0,4.0,4,3,1615.0,1.0,0.0,1.0,0,2,1.60
4,0,5,2.0,2.0,1,3,582.0,0.0,1.0,0.0,0,2,0.48


In [5]:
df.shape

(3554, 13)

In order to apply linear models
- one hot encoding
- scaling
- log transformation on Price col

In [6]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category
# Why are we doing this -> to apply linear models

X = df.drop(columns=['price'])
y = df['price']

In [20]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [21]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [22]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(),
         ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], remainder='passthrough'
)

In [23]:
# Creating a pipeline
# LinearRegression()
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [24]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
scores.mean()

0.8845360715052786

In [25]:
scores.std()

0.014784881452420021

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
X_train.shape

(2843, 12)

In [27]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [28]:
y_pred = pipeline.predict(X_test)

In [29]:
# since we have transformed y target col to log(y) 
# To convert it back to normal, we have done e^log(y)
y_pred = np.expm1(y_pred)

In [30]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(np.expm1(y_test), y_pred)

0.5324591082613233

on an average 0.64 lacs wrong for Lin Reg model

on an average 0.53 lacs wrong for SVR model


What we can do to improve to reduce this error
- different algorithm
- hyper-parameter tuning
- feature engineering
- more data